# Predicting housing price based on input features.

### Dataset:
* `California housing (form sklearn)`
### Model:
- `RandomForest`


Later will tunning the hyperparaemeters and use `MLFlow` for tracking

In [21]:
import pandas as pd
import numpy as np
import mlflow.sklearn
import mlflow
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [22]:
house_data = fetch_california_housing()

In [23]:
print(house_data)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [24]:
data = pd.DataFrame(house_data.data, columns=house_data.feature_names)

In [25]:
data["Price"] = house_data.target

In [26]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [27]:
# Independent and dependent features:
X=data.drop(columns=["Price"])
y= data["Price"]

In [28]:
# Split the data:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=42)

In [29]:
# Model schemas

from mlflow.models import infer_signature
signature=infer_signature(X_train, y_train)

In [30]:
# Hyperparameters:
params = {
    "n_estimators": [100,150,200],
    "max_depth":[5,10,None],
    "min_samples_split":[2,5],
    "min_samples_leaf":[1,2]
}

In [31]:
# Hyperparameters tunning:
def hyperparameter_tunning(X_train, y_train, params):
    grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, verbose=2, n_jobs=-1, scoring="neg_mean_squared_error",cv=4)
    grid_search.fit(X_train, y_train)
    return grid_search

In [34]:
from urllib.parse import urlparse
# Start with the MLFlow:
with mlflow.start_run():
    ## Tune the hyperparameters:
    grid_search=hyperparameter_tunning(X_train, y_train, params)
    
    ## Best model:
    best_model = grid_search.best_estimator_
    
    ## Evaluate the model:
    y_pred=best_model.predict(X_test)
    mse=mean_squared_error(y_pred=y_pred, y_true=y_test)
    
    ## Logging the model:
    mlflow.log_param("Best_n_estimator", grid_search.best_params_["n_estimators"])
    mlflow.log_param("Best_max_depth", grid_search.best_params_["max_depth"])
    mlflow.log_param("Best_min_samples_split", grid_search.best_params_["min_samples_split"])
    mlflow.log_param("Best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"])
    mlflow.log_metric("MSE", mse)
    
    ## Tracking URL:
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_uri_type=urlparse(url=mlflow.get_tracking_uri()).scheme
    if tracking_uri_type != "file":
        mlflow.sklearn.log_model(sk_model=best_model, registered_model_name="Best Randomforest Model")
    else:
        mlflow.sklearn.log_model(sk_model=best_model, signature=signature)
        
        
    print(f"Best Params: {grid_search.best_params_}")
    print(f"MSE: {mse}")

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   5.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   5.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.6s
[CV] END max_depth=5, min_samples_leaf

2025/11/24 19:42:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best Randomforest Model'.
2025/11/24 19:43:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Randomforest Model, version 1
Created version '1' of model 'Best Randomforest Model'.


Best Params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
MSE: 0.2547635211624389
🏃 View run bold-carp-297 at: http://127.0.0.1:5000/#/experiments/0/runs/43fbeb54bbaf42a4b8dd95fec27b41fe
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
